### Library

In [1]:
import pandas as pd
import matplotlib as plt
import seaborn as sns
import numpy as np

### Obtain

#### Inpatient Prospective Payment System (IPPS) Provider Summary for the Top 100 Diagnosis-Related Groups (DRG) - FY2011

https://data.cms.gov/Medicare-Inpatient/Inpatient-Prospective-Payment-System-IPPS-Provider/97k6-zzx3

In [2]:
data = pd.read_csv('Inpatient_Prospective_Payment_System__IPPS__Provider_Summary_for_the_Top_100_Diagnosis-Related_Groups__DRG__-_FY2011.csv')

In [3]:
data['DRG_id'] = data['DRG Definition'].apply(lambda x: x[:3])

In [4]:
data['DRG_label'] = data['DRG Definition'].apply(lambda x: x.split(' - ')[1].split(' W')[0])

In [5]:
data['without_ccmcc'] = data['DRG Definition'].apply(lambda x: 1 if ' W/O ' in x else 0)
data['with_mcc'] = data['DRG Definition'].apply(lambda x: 1 if ' W MCC' in x else 0)
data['with_cc'] = data['DRG Definition'].apply(lambda x: 1 if ' W CC'  in x else 0)
data['with_ccmcc'] = data['DRG Definition'].apply(lambda x: 1 if ' W CC/MCC'  in x else 0)
data.with_cc = data.with_cc - data.with_ccmcc

In [6]:
dict_transfer = {}
for i in data.DRG_id.unique():
    dict_transfer.update({i:data.loc[data.DRG_id == i][' Average Total Payments '].max()})


In [7]:
data['ratio_to_max_payment'] = data[' Average Total Payments ']/ data.DRG_id.apply(lambda x: dict_transfer[x])

In [8]:
dict_transfer = {}
for i in data.DRG_id.unique():
    dict_transfer.update({i:data.loc[data.DRG_id == i][' Total Discharges '].max()})


In [9]:
data['ratio_to_max_discharge'] = data[' Total Discharges ']/ data.DRG_id.apply(lambda x: dict_transfer[x])

In [10]:
transfer_dict = {}
for i in data.columns:
    j = i.replace('!!','_')
    j = j.replace(' ', '_')
    transfer_dict.update({i:j})
data = data.rename(columns = transfer_dict)


In [11]:
data['CountyState'] = data.County + data.Provider_State

#### Zip Code to County

In [12]:
zip_code_data = pd.read_csv('uszips.csv', usecols=[0,10,11])

In [13]:
a= pd.read_excel('ZIP_COUNTY_092019.xlsx', usecols=[0,1])

In [14]:
zip = a.merge(zip_code_data, how='inner', left_on='county', right_on='county_fips')[['zip_x', 'county_name']]
zip = zip.rename(columns={'zip_x':'zip'})

Code to get county if the CMS data and write it in the original csv.

```python
transfer_dict = {}
for i in data.Provider_Zip_Code.unique():
    if i in zip.zip.unique():
        continue
    else:
        
        x = input(prompt=i)
        transfer_dict.update({i:x})

transfer_dict.update(zip.set_index('zip').to_dict()['county_name'])
        
a = pd.read_csv('Inpatient_Prospective_Payment_System__IPPS__Provider_Summary_for_the_Top_100_Diagnosis-Related_Groups__DRG__-_FY2011.csv')
a['County'] = a["Provider Zip Code"].apply(lambda x: transfer_dict[x])
a.to_csv('Inpatient_Prospective_Payment_System__IPPS__Provider_Summary_for_the_Top_100_Diagnosis-Related_Groups__DRG__-_FY2011.csv')
```

### American Community Study

In [15]:
acs_data = pd.read_csv('ACSDP5Y2017.DP05_data_with_overlays_2019-12-16T161924.csv',skiprows = 1)
acs_income_data = pd.read_csv('ACSST5Y2017.S1701_data_with_overlays_2019-12-17T171021.csv', skiprows = 1)
acs_income2_data = pd.read_csv('ACSST1Y2012.S1901_data_with_overlays_2019-12-21T221020.csv', skiprows=1)

C:\Users\FlatIron_User\.conda\envs\learn-env\lib\site-packages\IPython\core\interactiveshell.py:2785: DtypeWarning: Columns (12,13,24,25,30,31,126,127,324,325,330,331,336,337,342,343) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [16]:
i

'ratio_to_max_discharge'

In [17]:
for i in acs_data.columns:
    if 'Error' in i:
        acs_data = acs_data.drop(columns = [i])

for i in acs_income_data.columns:
    if 'Error' in i:
        acs_income_data = acs_income_data.drop(columns = [i])
        
for i in acs_income2_data.columns:
    if 'Error' in i:
        acs_income2_data = acs_income2_data.drop(columns = [i])
        
for i in acs_income2_data.columns:
    if (acs_income2_data[i].astype(str) == '(X)').sum():
        acs_income2_data = acs_income2_data.drop(columns = [i])    

In [18]:
acs_data['County'] = acs_data['Geographic Area Name'].apply(lambda x: x.split(',')[0][:-7])
acs_data['State'] = acs_data['Geographic Area Name'].apply(lambda x: x.split(',')[1])

In [19]:
state_abb ={"Alabama":"AL",
"Alaska":"AK",
"Arizona":"AZ",
"Arkansas":"AR",
"California":"CA",
"Colorado":"CO",
"Connecticut":"CT",
"Delaware":"DE",
"Florida":"FL",
"Georgia":"GA",
"Hawaii":"HI",
"Idaho":"ID",
"Illinois":"IL",
"Indiana":"IN",
"Iowa":"IA",
"Kansas":"KS",
"Kentucky":"KY",
"Louisiana":"LA",
"Maine":"ME",
"Maryland":"MD",
"Massachusetts":"MA",
"Michigan":"MI",
"Minnesota":"MN",
"Mississippi":"MS",
"Missouri":"MO",
"Montana":"MT",
"Nebraska":"NE",
"Nevada":"NV",
"New Hampshire":"NH",
"New Jersey":"NJ",
"New Mexico":"NM",
"New York":"NY",
"North Carolina":"NC",
"North Dakota":"ND",
"Ohio":"OH",
"Oklahoma":"OK",
"Oregon":"OR",
"Pennsylvania":"PA",
"Rhode Island":"RI",
"South Carolina":"SC",
"South Dakota":"SD",
"Tennessee":"TN",
"Texas":"TX",
"Utah":"UT",
"Vermont":"VT",
"Virginia":"VA",
"Washington":"WA",
"West Virginia":"WV",
"Wisconsin":"WI",
"Wyoming":"WY",
"Puerto Rico":"PR",
           "District of Columbia":"DC"}



In [20]:
acs_data.State = acs_data.State.apply(lambda x: state_abb[x[1:]])

In [21]:
acs_data['CountyState'] = acs_data.County + acs_data.State

In [23]:
acs_income_data = acs_income_data.drop(columns='Geographic Area Name')
acs_income2_data = acs_income2_data.drop(columns='Geographic Area Name')
acs_income_data = acs_income_data.merge(acs_income2_data, on= 'id')
acs_data = acs_data.merge(acs_income_data, on= 'id')

In [24]:
transfer_dict = {}
for i in acs_data.columns:
    j = i.replace('!!',' ')
    j = j.replace(' ','_')
    j = j.replace('Estimate_','')
    transfer_dict.update({i:j})

acs_data = acs_data.rename(columns=transfer_dict)
    
acs_data.head()

,id,Geographic_Area_Name,SEX_AND_AGE_Total_population,Percent_SEX_AND_AGE_Total_population,SEX_AND_AGE_Total_population_Male,Percent_SEX_AND_AGE_Total_population_Male,SEX_AND_AGE_Total_population_Female,Percent_SEX_AND_AGE_Total_population_Female,SEX_AND_AGE_Total_population_Sex_ratio_(males_per_100_females),Percent_SEX_AND_AGE_Total_population_Sex_ratio_(males_per_100_females),...,"Nonfamily_households_$25,000_to_$34,999","Nonfamily_households_$50,000_to_$74,999","Nonfamily_households_$75,000_to_$99,999","Nonfamily_households_$100,000_to_$149,999","Nonfamily_households_$150,000_to_$199,999","Nonfamily_households_$200,000_or_more",Nonfamily_households_Median_income_(dollars),Nonfamily_households_Mean_income_(dollars),Nonfamily_households_PERCENT_IMPUTED_Nonfamily_income_in_the_past_12_months,"Nonfamily_households_$35,000_to_$49,999"
0,0500000US01073,"Jefferson County, Alabama",659460,659460,312333,47.4,347127,52.6,90.0,(X),...,13.8,14.9,5.4,3.0,0.8,0.9,28077,38211,28.3,15.3
1,0500000US01049,"DeKalb County, Alabama",71194,71194,35328,49.6,35866,50.4,98.5,(X),...,10.5,6.3,3.1,1.4,0.0,0.0,17367,24933,25.9,13.8
2,0500000US01081,"Lee County, Alabama",156597,156597,77172,49.3,79425,50.7,97.2,(X),...,8.4,15.8,4.0,3.3,0.0,0.0,21746,30008,25.8,11.1
3,0500000US01097,"Mobile County, Alabama",414328,414328,197968,47.8,216360,52.2,91.5,(X),...,12.2,15.5,4.5,1.9,0.2,0.6,24081,32725,21.3,14.0
4,0500000US01121,"Talladega County, Alabama",80888,80888,39140,48.4,41748,51.6,93.8,(X),...,12.5,6.9,4.0,1.0,0.0,0.2,15849,23878,34.1,7.2


### Merging 

In [25]:
df = acs_data.merge(data, on='CountyState')

In [26]:
drop_columns = ['Unnamed:_0','SEX_AND_AGE_Total_population_Male',
                'Percent_SEX_AND_AGE_Total_population',
'SEX_AND_AGE_Total_population_Female',
'SEX_AND_AGE_Total_population_Under_5_years',
'SEX_AND_AGE_Total_population_5_to_9_years',
'SEX_AND_AGE_Total_population_10_to_14_years',
'SEX_AND_AGE_Total_population_15_to_19_years',
'SEX_AND_AGE_Total_population_20_to_24_years',
'SEX_AND_AGE_Total_population_25_to_34_years',
'SEX_AND_AGE_Total_population_35_to_44_years',
'SEX_AND_AGE_Total_population_45_to_54_years',
'SEX_AND_AGE_Total_population_55_to_59_years',
'SEX_AND_AGE_Total_population_60_to_64_years',
'SEX_AND_AGE_Total_population_65_to_74_years',
'SEX_AND_AGE_Total_population_75_to_84_years',
'SEX_AND_AGE_Total_population_85_years_and_over',
'SEX_AND_AGE_Total_population_Under_18_years',
'SEX_AND_AGE_Total_population_16_years_and_over',
'SEX_AND_AGE_Total_population_18_years_and_over',
'SEX_AND_AGE_Total_population_21_years_and_over',
'SEX_AND_AGE_Total_population_62_years_and_over',
'SEX_AND_AGE_Total_population_65_years_and_over',
'SEX_AND_AGE_Total_population_18_years_and_over.1',
'SEX_AND_AGE_Total_population_18_years_and_over_Male',
'SEX_AND_AGE_Total_population_18_years_and_over_Female',
'SEX_AND_AGE_Total_population_65_years_and_over.1',
'SEX_AND_AGE_Total_population_65_years_and_over_Male',
'SEX_AND_AGE_Total_population_65_years_and_over_Female']
df = df.drop(columns=drop_columns)

In [27]:
drop_columns = ['Percent_SEX_AND_AGE_Total_population_Sex_ratio_(males_per_100_females)',
'Percent_SEX_AND_AGE_Total_population_Median_age_(years)',
'Percent_SEX_AND_AGE_Total_population_18_years_and_over_Sex_ratio_(males_per_100_females)',
'Percent_SEX_AND_AGE_Total_population_65_years_and_over_Sex_ratio_(males_per_100_females)']
df = df.drop(columns=drop_columns)

In [28]:
drop_columns = ['RACE_Total_population',
'RACE_Total_population_One_race',
'RACE_Total_population_Two_or_more_races',
'RACE_Total_population_One_race.1',
'RACE_Total_population_One_race_White',
'RACE_Total_population_One_race_Black_or_African_American',
'RACE_Total_population_One_race_American_Indian_and_Alaska_Native',
'RACE_Total_population_One_race_American_Indian_and_Alaska_Native_Cherokee_tribal_grouping',
'RACE_Total_population_One_race_American_Indian_and_Alaska_Native_Chippewa_tribal_grouping',
'RACE_Total_population_One_race_American_Indian_and_Alaska_Native_Navajo_tribal_grouping',
'RACE_Total_population_One_race_American_Indian_and_Alaska_Native_Sioux_tribal_grouping',
'RACE_Total_population_One_race_Asian',
'RACE_Total_population_One_race_Asian_Asian_Indian',
'RACE_Total_population_One_race_Asian_Chinese',
'RACE_Total_population_One_race_Asian_Filipino',
'RACE_Total_population_One_race_Asian_Japanese',
'RACE_Total_population_One_race_Asian_Korean',
'RACE_Total_population_One_race_Asian_Vietnamese',
'RACE_Total_population_One_race_Asian_Other_Asian',
'RACE_Total_population_One_race_Native_Hawaiian_and_Other_Pacific_Islander',
'RACE_Total_population_One_race_Native_Hawaiian_and_Other_Pacific_Islander_Native_Hawaiian',
'RACE_Total_population_One_race_Native_Hawaiian_and_Other_Pacific_Islander_Guamanian_or_Chamorro',
'RACE_Total_population_One_race_Native_Hawaiian_and_Other_Pacific_Islander_Samoan',
'RACE_Total_population_One_race_Native_Hawaiian_and_Other_Pacific_Islander_Other_Pacific_Islander',
'RACE_Total_population_One_race_Some_other_race',
'RACE_Total_population_Two_or_more_races.1',
'RACE_Total_population_Two_or_more_races_White_and_Black_or_African_American',
'RACE_Total_population_Two_or_more_races_White_and_American_Indian_and_Alaska_Native',
'RACE_Total_population_Two_or_more_races_White_and_Asian',
'RACE_Total_population_Two_or_more_races_Black_or_African_American_and_American_Indian_and_Alaska_Native']
df = df.drop(columns=drop_columns)

In [29]:
drop_columns = ['Percent_RACE_Total_population',
'Race_alone_or_in_combination_with_one_or_more_other_races_Total_population',
'Race_alone_or_in_combination_with_one_or_more_other_races_Total_population_White',
'Race_alone_or_in_combination_with_one_or_more_other_races_Total_population_Black_or_African_American',
'Race_alone_or_in_combination_with_one_or_more_other_races_Total_population_American_Indian_and_Alaska_Native',
'Race_alone_or_in_combination_with_one_or_more_other_races_Total_population_Asian',
'Race_alone_or_in_combination_with_one_or_more_other_races_Total_population_Native_Hawaiian_and_Other_Pacific_Islander',
'Race_alone_or_in_combination_with_one_or_more_other_races_Total_population_Some_other_race']
df = df.drop(columns=drop_columns)

In [30]:
drop_columns = ['HISPANIC_OR_LATINO_AND_RACE_Total_population',
'HISPANIC_OR_LATINO_AND_RACE_Total_population_Hispanic_or_Latino_(of_any_race)',
'HISPANIC_OR_LATINO_AND_RACE_Total_population_Hispanic_or_Latino_(of_any_race)_Mexican',
'HISPANIC_OR_LATINO_AND_RACE_Total_population_Hispanic_or_Latino_(of_any_race)_Puerto_Rican',
'HISPANIC_OR_LATINO_AND_RACE_Total_population_Hispanic_or_Latino_(of_any_race)_Cuban',
'HISPANIC_OR_LATINO_AND_RACE_Total_population_Hispanic_or_Latino_(of_any_race)_Other_Hispanic_or_Latino',
'HISPANIC_OR_LATINO_AND_RACE_Total_population_Not_Hispanic_or_Latino',
'HISPANIC_OR_LATINO_AND_RACE_Total_population_Not_Hispanic_or_Latino_White_alone',
'HISPANIC_OR_LATINO_AND_RACE_Total_population_Not_Hispanic_or_Latino_Black_or_African_American_alone',
'HISPANIC_OR_LATINO_AND_RACE_Total_population_Not_Hispanic_or_Latino_American_Indian_and_Alaska_Native_alone',
'HISPANIC_OR_LATINO_AND_RACE_Total_population_Not_Hispanic_or_Latino_Asian_alone',
'HISPANIC_OR_LATINO_AND_RACE_Total_population_Not_Hispanic_or_Latino_Native_Hawaiian_and_Other_Pacific_Islander_alone',
'HISPANIC_OR_LATINO_AND_RACE_Total_population_Not_Hispanic_or_Latino_Some_other_race_alone',
'HISPANIC_OR_LATINO_AND_RACE_Total_population_Not_Hispanic_or_Latino_Two_or_more_races',
'HISPANIC_OR_LATINO_AND_RACE_Total_population_Not_Hispanic_or_Latino_Two_or_more_races_Two_races_including_Some_other_race',
'HISPANIC_OR_LATINO_AND_RACE_Total_population_Not_Hispanic_or_Latino_Two_or_more_races_Two_races_excluding_Some_other_race_and_Three_or_more_races']
df = df.drop(columns=drop_columns)


In [31]:
drop_columns= ['Percent_Total_housing_units',
              'CITIZEN_VOTING_AGE_POPULATION_Citizen_18_and_over_population',
'CITIZEN_VOTING_AGE_POPULATION_Citizen_18_and_over_population_Male',
'CITIZEN_VOTING_AGE_POPULATION_Citizen_18_and_over_population_Female',
              'County_x',
               'Total_Population_for_whom_poverty_status_is_determined',
'Total_Population_for_whom_poverty_status_is_determined_AGE_Under_18_years',
'Total_Population_for_whom_poverty_status_is_determined_AGE_Under_18_years_Under_5_years',
'Total_Population_for_whom_poverty_status_is_determined_AGE_Under_18_years_5_to_17_years',
'Total_Population_for_whom_poverty_status_is_determined_AGE_Under_18_years_Related_children_of_householder_under_18_years',
'Total_Population_for_whom_poverty_status_is_determined_AGE_18_to_64_years',
'Total_Population_for_whom_poverty_status_is_determined_AGE_18_to_64_years_18_to_34_years',
'Total_Population_for_whom_poverty_status_is_determined_AGE_18_to_64_years_35_to_64_years',
'Total_Population_for_whom_poverty_status_is_determined_AGE_60_years_and_over',
'Total_Population_for_whom_poverty_status_is_determined_AGE_65_years_and_over',
'Total_Population_for_whom_poverty_status_is_determined_SEX_Male',
'Total_Population_for_whom_poverty_status_is_determined_SEX_Female',
'Total_Population_for_whom_poverty_status_is_determined_RACE_AND_HISPANIC_OR_LATINO_ORIGIN_White_alone',
'Total_Population_for_whom_poverty_status_is_determined_RACE_AND_HISPANIC_OR_LATINO_ORIGIN_Black_or_African_American_alone',
'Total_Population_for_whom_poverty_status_is_determined_RACE_AND_HISPANIC_OR_LATINO_ORIGIN_American_Indian_and_Alaska_Native_alone',
'Total_Population_for_whom_poverty_status_is_determined_RACE_AND_HISPANIC_OR_LATINO_ORIGIN_Asian_alone',
'Total_Population_for_whom_poverty_status_is_determined_RACE_AND_HISPANIC_OR_LATINO_ORIGIN_Native_Hawaiian_and_Other_Pacific_Islander_alone',
'Total_Population_for_whom_poverty_status_is_determined_RACE_AND_HISPANIC_OR_LATINO_ORIGIN_Some_other_race_alone',
'Total_Population_for_whom_poverty_status_is_determined_RACE_AND_HISPANIC_OR_LATINO_ORIGIN_Two_or_more_races',
'Total_Population_for_whom_poverty_status_is_determined_RACE_AND_HISPANIC_OR_LATINO_ORIGIN_Hispanic_or_Latino_origin_(of_any_race)',
'Total_Population_for_whom_poverty_status_is_determined_RACE_AND_HISPANIC_OR_LATINO_ORIGIN_White_alone,_not_Hispanic_or_Latino',
'Total_Population_for_whom_poverty_status_is_determined_EDUCATIONAL_ATTAINMENT_Population_25_years_and_over',
'Total_Population_for_whom_poverty_status_is_determined_EDUCATIONAL_ATTAINMENT_Population_25_years_and_over_Less_than_high_school_graduate',
'Total_Population_for_whom_poverty_status_is_determined_EDUCATIONAL_ATTAINMENT_Population_25_years_and_over_High_school_graduate_(includes_equivalency)',
"Total_Population_for_whom_poverty_status_is_determined_EDUCATIONAL_ATTAINMENT_Population_25_years_and_over_Some_college,_associate's_degree",
"Total_Population_for_whom_poverty_status_is_determined_EDUCATIONAL_ATTAINMENT_Population_25_years_and_over_Bachelor's_degree_or_higher",
'Total_Population_for_whom_poverty_status_is_determined_EMPLOYMENT_STATUS_Civilian_labor_force_16_years_and_over',
'Total_Population_for_whom_poverty_status_is_determined_EMPLOYMENT_STATUS_Civilian_labor_force_16_years_and_over_Employed',
'Total_Population_for_whom_poverty_status_is_determined_EMPLOYMENT_STATUS_Civilian_labor_force_16_years_and_over_Employed_Male',
'Total_Population_for_whom_poverty_status_is_determined_EMPLOYMENT_STATUS_Civilian_labor_force_16_years_and_over_Employed_Female',
'Total_Population_for_whom_poverty_status_is_determined_EMPLOYMENT_STATUS_Civilian_labor_force_16_years_and_over_Unemployed',
'Total_Population_for_whom_poverty_status_is_determined_EMPLOYMENT_STATUS_Civilian_labor_force_16_years_and_over_Unemployed_Male',
'Total_Population_for_whom_poverty_status_is_determined_EMPLOYMENT_STATUS_Civilian_labor_force_16_years_and_over_Unemployed_Female',
'Total_Population_for_whom_poverty_status_is_determined_WORK_EXPERIENCE_Population_16_years_and_over',
'Total_Population_for_whom_poverty_status_is_determined_WORK_EXPERIENCE_Population_16_years_and_over_Worked_full-time,_year-round_in_the_past_12_months',
'Total_Population_for_whom_poverty_status_is_determined_WORK_EXPERIENCE_Population_16_years_and_over_Worked_part-time_or_part-year_in_the_past_12_months',
'Total_Population_for_whom_poverty_status_is_determined_WORK_EXPERIENCE_Population_16_years_and_over_Did_not_work',
'Total_Population_for_whom_poverty_status_is_determined_ALL_INDIVIDUALS_WITH_INCOME_BELOW_THE_FOLLOWING_POVERTY_RATIOS_50_percent_of_poverty_level',
'Total_Population_for_whom_poverty_status_is_determined_ALL_INDIVIDUALS_WITH_INCOME_BELOW_THE_FOLLOWING_POVERTY_RATIOS_125_percent_of_poverty_level',
'Total_Population_for_whom_poverty_status_is_determined_ALL_INDIVIDUALS_WITH_INCOME_BELOW_THE_FOLLOWING_POVERTY_RATIOS_150_percent_of_poverty_level',
'Total_Population_for_whom_poverty_status_is_determined_ALL_INDIVIDUALS_WITH_INCOME_BELOW_THE_FOLLOWING_POVERTY_RATIOS_185_percent_of_poverty_level',
'Total_Population_for_whom_poverty_status_is_determined_ALL_INDIVIDUALS_WITH_INCOME_BELOW_THE_FOLLOWING_POVERTY_RATIOS_200_percent_of_poverty_level',
'Total_Population_for_whom_poverty_status_is_determined_ALL_INDIVIDUALS_WITH_INCOME_BELOW_THE_FOLLOWING_POVERTY_RATIOS_300_percent_of_poverty_level',
'Total_Population_for_whom_poverty_status_is_determined_ALL_INDIVIDUALS_WITH_INCOME_BELOW_THE_FOLLOWING_POVERTY_RATIOS_400_percent_of_poverty_level',
'Total_Population_for_whom_poverty_status_is_determined_ALL_INDIVIDUALS_WITH_INCOME_BELOW_THE_FOLLOWING_POVERTY_RATIOS_500_percent_of_poverty_level',
'Total_UNRELATED_INDIVIDUALS_FOR_WHOM_POVERTY_STATUS_IS_DETERMINED',
'Total_UNRELATED_INDIVIDUALS_FOR_WHOM_POVERTY_STATUS_IS_DETERMINED_Male',
'Total_UNRELATED_INDIVIDUALS_FOR_WHOM_POVERTY_STATUS_IS_DETERMINED_Female',
'Total_UNRELATED_INDIVIDUALS_FOR_WHOM_POVERTY_STATUS_IS_DETERMINED_15_years',
'Total_UNRELATED_INDIVIDUALS_FOR_WHOM_POVERTY_STATUS_IS_DETERMINED_16_to_17_years',
'Total_UNRELATED_INDIVIDUALS_FOR_WHOM_POVERTY_STATUS_IS_DETERMINED_18_to_24_years',
'Total_UNRELATED_INDIVIDUALS_FOR_WHOM_POVERTY_STATUS_IS_DETERMINED_25_to_34_years',
'Total_UNRELATED_INDIVIDUALS_FOR_WHOM_POVERTY_STATUS_IS_DETERMINED_35_to_44_years',
'Total_UNRELATED_INDIVIDUALS_FOR_WHOM_POVERTY_STATUS_IS_DETERMINED_45_to_54_years',
'Total_UNRELATED_INDIVIDUALS_FOR_WHOM_POVERTY_STATUS_IS_DETERMINED_55_to_64_years',
'Total_UNRELATED_INDIVIDUALS_FOR_WHOM_POVERTY_STATUS_IS_DETERMINED_65_to_74_years',
'Total_UNRELATED_INDIVIDUALS_FOR_WHOM_POVERTY_STATUS_IS_DETERMINED_75_years_and_over',
'Total_UNRELATED_INDIVIDUALS_FOR_WHOM_POVERTY_STATUS_IS_DETERMINED_Mean_income_deficit_for_unrelated_individuals_(dollars)',
'Total_UNRELATED_INDIVIDUALS_FOR_WHOM_POVERTY_STATUS_IS_DETERMINED_Worked_full-time,_year-round_in_the_past_12_months',
'Total_UNRELATED_INDIVIDUALS_FOR_WHOM_POVERTY_STATUS_IS_DETERMINED_Worked_less_than_full-time,_year-round_in_the_past_12_months',
'Total_UNRELATED_INDIVIDUALS_FOR_WHOM_POVERTY_STATUS_IS_DETERMINED_Did_not_work']
df = df.drop(columns=drop_columns)


In [32]:
drop_columns =['Below_poverty_level_Population_for_whom_poverty_status_is_determined',
'Below_poverty_level_Population_for_whom_poverty_status_is_determined_AGE_Under_18_years',
'Below_poverty_level_Population_for_whom_poverty_status_is_determined_AGE_Under_18_years_Under_5_years',
'Below_poverty_level_Population_for_whom_poverty_status_is_determined_AGE_Under_18_years_5_to_17_years',
'Below_poverty_level_Population_for_whom_poverty_status_is_determined_AGE_Under_18_years_Related_children_of_householder_under_18_years',
'Below_poverty_level_Population_for_whom_poverty_status_is_determined_AGE_18_to_64_years',
'Below_poverty_level_Population_for_whom_poverty_status_is_determined_AGE_18_to_64_years_18_to_34_years',
'Below_poverty_level_Population_for_whom_poverty_status_is_determined_AGE_18_to_64_years_35_to_64_years',
'Below_poverty_level_Population_for_whom_poverty_status_is_determined_AGE_60_years_and_over',
'Below_poverty_level_Population_for_whom_poverty_status_is_determined_AGE_65_years_and_over',
'Below_poverty_level_Population_for_whom_poverty_status_is_determined_SEX_Male',
'Below_poverty_level_Population_for_whom_poverty_status_is_determined_SEX_Female',
'Below_poverty_level_Population_for_whom_poverty_status_is_determined_RACE_AND_HISPANIC_OR_LATINO_ORIGIN_White_alone',
'Below_poverty_level_Population_for_whom_poverty_status_is_determined_RACE_AND_HISPANIC_OR_LATINO_ORIGIN_Black_or_African_American_alone',
'Below_poverty_level_Population_for_whom_poverty_status_is_determined_RACE_AND_HISPANIC_OR_LATINO_ORIGIN_American_Indian_and_Alaska_Native_alone',
'Below_poverty_level_Population_for_whom_poverty_status_is_determined_RACE_AND_HISPANIC_OR_LATINO_ORIGIN_Asian_alone',
'Below_poverty_level_Population_for_whom_poverty_status_is_determined_RACE_AND_HISPANIC_OR_LATINO_ORIGIN_Native_Hawaiian_and_Other_Pacific_Islander_alone',
'Below_poverty_level_Population_for_whom_poverty_status_is_determined_RACE_AND_HISPANIC_OR_LATINO_ORIGIN_Some_other_race_alone',
'Below_poverty_level_Population_for_whom_poverty_status_is_determined_RACE_AND_HISPANIC_OR_LATINO_ORIGIN_Two_or_more_races',
'Below_poverty_level_Population_for_whom_poverty_status_is_determined_RACE_AND_HISPANIC_OR_LATINO_ORIGIN_Hispanic_or_Latino_origin_(of_any_race)',
'Below_poverty_level_Population_for_whom_poverty_status_is_determined_RACE_AND_HISPANIC_OR_LATINO_ORIGIN_White_alone,_not_Hispanic_or_Latino',
'Below_poverty_level_Population_for_whom_poverty_status_is_determined_EDUCATIONAL_ATTAINMENT_Population_25_years_and_over',
'Below_poverty_level_Population_for_whom_poverty_status_is_determined_EDUCATIONAL_ATTAINMENT_Population_25_years_and_over_Less_than_high_school_graduate',
'Below_poverty_level_Population_for_whom_poverty_status_is_determined_EDUCATIONAL_ATTAINMENT_Population_25_years_and_over_High_school_graduate_(includes_equivalency)',
"Below_poverty_level_Population_for_whom_poverty_status_is_determined_EDUCATIONAL_ATTAINMENT_Population_25_years_and_over_Some_college,_associate's_degree",
"Below_poverty_level_Population_for_whom_poverty_status_is_determined_EDUCATIONAL_ATTAINMENT_Population_25_years_and_over_Bachelor's_degree_or_higher",
'Below_poverty_level_Population_for_whom_poverty_status_is_determined_EMPLOYMENT_STATUS_Civilian_labor_force_16_years_and_over',
'Below_poverty_level_Population_for_whom_poverty_status_is_determined_EMPLOYMENT_STATUS_Civilian_labor_force_16_years_and_over_Employed',
'Below_poverty_level_Population_for_whom_poverty_status_is_determined_EMPLOYMENT_STATUS_Civilian_labor_force_16_years_and_over_Employed_Male',
'Below_poverty_level_Population_for_whom_poverty_status_is_determined_EMPLOYMENT_STATUS_Civilian_labor_force_16_years_and_over_Employed_Female',
'Below_poverty_level_Population_for_whom_poverty_status_is_determined_EMPLOYMENT_STATUS_Civilian_labor_force_16_years_and_over_Unemployed',
'Below_poverty_level_Population_for_whom_poverty_status_is_determined_EMPLOYMENT_STATUS_Civilian_labor_force_16_years_and_over_Unemployed_Male',
'Below_poverty_level_Population_for_whom_poverty_status_is_determined_EMPLOYMENT_STATUS_Civilian_labor_force_16_years_and_over_Unemployed_Female',
'Below_poverty_level_Population_for_whom_poverty_status_is_determined_WORK_EXPERIENCE_Population_16_years_and_over',
'Below_poverty_level_Population_for_whom_poverty_status_is_determined_WORK_EXPERIENCE_Population_16_years_and_over_Worked_full-time,_year-round_in_the_past_12_months',
'Below_poverty_level_Population_for_whom_poverty_status_is_determined_WORK_EXPERIENCE_Population_16_years_and_over_Worked_part-time_or_part-year_in_the_past_12_months',
'Below_poverty_level_Population_for_whom_poverty_status_is_determined_WORK_EXPERIENCE_Population_16_years_and_over_Did_not_work',
'Below_poverty_level_Population_for_whom_poverty_status_is_determined_ALL_INDIVIDUALS_WITH_INCOME_BELOW_THE_FOLLOWING_POVERTY_RATIOS_50_percent_of_poverty_level',
'Below_poverty_level_Population_for_whom_poverty_status_is_determined_ALL_INDIVIDUALS_WITH_INCOME_BELOW_THE_FOLLOWING_POVERTY_RATIOS_125_percent_of_poverty_level',
'Below_poverty_level_Population_for_whom_poverty_status_is_determined_ALL_INDIVIDUALS_WITH_INCOME_BELOW_THE_FOLLOWING_POVERTY_RATIOS_150_percent_of_poverty_level',
'Below_poverty_level_Population_for_whom_poverty_status_is_determined_ALL_INDIVIDUALS_WITH_INCOME_BELOW_THE_FOLLOWING_POVERTY_RATIOS_185_percent_of_poverty_level',
'Below_poverty_level_Population_for_whom_poverty_status_is_determined_ALL_INDIVIDUALS_WITH_INCOME_BELOW_THE_FOLLOWING_POVERTY_RATIOS_200_percent_of_poverty_level',
'Below_poverty_level_Population_for_whom_poverty_status_is_determined_ALL_INDIVIDUALS_WITH_INCOME_BELOW_THE_FOLLOWING_POVERTY_RATIOS_300_percent_of_poverty_level',
'Below_poverty_level_Population_for_whom_poverty_status_is_determined_ALL_INDIVIDUALS_WITH_INCOME_BELOW_THE_FOLLOWING_POVERTY_RATIOS_400_percent_of_poverty_level',
'Below_poverty_level_Population_for_whom_poverty_status_is_determined_ALL_INDIVIDUALS_WITH_INCOME_BELOW_THE_FOLLOWING_POVERTY_RATIOS_500_percent_of_poverty_level',
'Below_poverty_level_UNRELATED_INDIVIDUALS_FOR_WHOM_POVERTY_STATUS_IS_DETERMINED',
'Below_poverty_level_UNRELATED_INDIVIDUALS_FOR_WHOM_POVERTY_STATUS_IS_DETERMINED_Male',
'Below_poverty_level_UNRELATED_INDIVIDUALS_FOR_WHOM_POVERTY_STATUS_IS_DETERMINED_Female',
'Below_poverty_level_UNRELATED_INDIVIDUALS_FOR_WHOM_POVERTY_STATUS_IS_DETERMINED_15_years',
'Below_poverty_level_UNRELATED_INDIVIDUALS_FOR_WHOM_POVERTY_STATUS_IS_DETERMINED_16_to_17_years',
'Below_poverty_level_UNRELATED_INDIVIDUALS_FOR_WHOM_POVERTY_STATUS_IS_DETERMINED_18_to_24_years',
'Below_poverty_level_UNRELATED_INDIVIDUALS_FOR_WHOM_POVERTY_STATUS_IS_DETERMINED_25_to_34_years',
'Below_poverty_level_UNRELATED_INDIVIDUALS_FOR_WHOM_POVERTY_STATUS_IS_DETERMINED_35_to_44_years',
'Below_poverty_level_UNRELATED_INDIVIDUALS_FOR_WHOM_POVERTY_STATUS_IS_DETERMINED_45_to_54_years',
'Below_poverty_level_UNRELATED_INDIVIDUALS_FOR_WHOM_POVERTY_STATUS_IS_DETERMINED_55_to_64_years',
'Below_poverty_level_UNRELATED_INDIVIDUALS_FOR_WHOM_POVERTY_STATUS_IS_DETERMINED_65_to_74_years',
'Below_poverty_level_UNRELATED_INDIVIDUALS_FOR_WHOM_POVERTY_STATUS_IS_DETERMINED_75_years_and_over',
'Below_poverty_level_UNRELATED_INDIVIDUALS_FOR_WHOM_POVERTY_STATUS_IS_DETERMINED_Mean_income_deficit_for_unrelated_individuals_(dollars)',
'Below_poverty_level_UNRELATED_INDIVIDUALS_FOR_WHOM_POVERTY_STATUS_IS_DETERMINED_Worked_full-time,_year-round_in_the_past_12_months',
'Below_poverty_level_UNRELATED_INDIVIDUALS_FOR_WHOM_POVERTY_STATUS_IS_DETERMINED_Worked_less_than_full-time,_year-round_in_the_past_12_months',
'Below_poverty_level_UNRELATED_INDIVIDUALS_FOR_WHOM_POVERTY_STATUS_IS_DETERMINED_Did_not_work',
              'County_y']
df = df.drop(columns=drop_columns)

In [33]:
for i in df.columns:
    print(f"'{i}',")

'id',
'Geographic_Area_Name',
'SEX_AND_AGE_Total_population',
'Percent_SEX_AND_AGE_Total_population_Male',
'Percent_SEX_AND_AGE_Total_population_Female',
'SEX_AND_AGE_Total_population_Sex_ratio_(males_per_100_females)',
'Percent_SEX_AND_AGE_Total_population_Under_5_years',
'Percent_SEX_AND_AGE_Total_population_5_to_9_years',
'Percent_SEX_AND_AGE_Total_population_10_to_14_years',
'Percent_SEX_AND_AGE_Total_population_15_to_19_years',
'Percent_SEX_AND_AGE_Total_population_20_to_24_years',
'Percent_SEX_AND_AGE_Total_population_25_to_34_years',
'Percent_SEX_AND_AGE_Total_population_35_to_44_years',
'Percent_SEX_AND_AGE_Total_population_45_to_54_years',
'Percent_SEX_AND_AGE_Total_population_55_to_59_years',
'Percent_SEX_AND_AGE_Total_population_60_to_64_years',
'Percent_SEX_AND_AGE_Total_population_65_to_74_years',
'Percent_SEX_AND_AGE_Total_population_75_to_84_years',
'Percent_SEX_AND_AGE_Total_population_85_years_and_over',
'SEX_AND_AGE_Total_population_Median_age_(years)',
'Percent_SEX_A

In [34]:
df.to_csv('obtain.csv')